In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [2]:
import findspark
findspark.find()

'/home/oem/spark'

In [3]:
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
# from pyspark import StorageLevel
from pyspark.sql import Window
from pyspark.sql import functions as f
# from pyspark.sql import SQLContext
# from pyspark import SparkContext

import pandas as pd
import math
import numpy as np
import scipy
from scipy.stats import chi2_contingency

In [4]:
spark = (SparkSession.builder
         .appName('ab_experiment')
         .config('spark.executor.cores', 1)
         .getOrCreate()
        )
print('http://localhost:4040')

21/08/19 00:41:17 WARN Utils: Your hostname, as-ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.8.107 instead (on interface wlp2s0)
21/08/19 00:41:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/08/19 00:41:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


http://localhost:4040


In [5]:
test_data_pd = pd.read_csv('datasets/test_data.csv')
viewer_data_pd = pd.read_csv('datasets/viewer_data.csv')

In [6]:
viewer_data_pd.head()

,viewer_id,gender,age,city
0,1918165,Female,39,Dallas
1,27662619,Female,28,New York
2,5493662,Female,53,Detroit
3,14441247,Male,41,New York
4,25595927,Male,53,Seattle


In [7]:
viewer_data_pd.count()

viewer_id    417464
gender       417464
age          417464
city         417464
dtype: int64

In [8]:
# any duplicates in dictionary?
viewer_data_pd.drop_duplicates().count()

viewer_id    417464
gender       417464
age          417464
city         417464
dtype: int64

In [9]:
test_data_pd.head()

,viewer_id,date,tv_make,tv_size,uhd_capable,tv_provider,total_time_watched,watched,test
0,24726768,2018-01-16,Sony,70,0,Comcast,10.75,0,1
1,25001464,2018-01-18,Sony,32,0,NaN,2.75,0,0
2,28291998,2018-01-18,Sony,50,1,Dish Network,20.00,0,0
3,17057157,2018-01-19,Sony,32,0,Comcast,1.50,0,0
4,29504447,2018-01-17,Sony,32,0,Comcast,17.50,0,0


In [10]:
test_data_pd.count()

viewer_id             418026
date                  418026
tv_make               418026
tv_size               418026
uhd_capable           418026
tv_provider           365306
total_time_watched    418026
watched               418026
test                  418026
dtype: int64

In [11]:
# any duplicates?
test_data_pd.drop_duplicates().count()

viewer_id             418026
date                  418026
tv_make               418026
tv_size               418026
uhd_capable           418026
tv_provider           365306
total_time_watched    418026
watched               418026
test                  418026
dtype: int64

In [12]:
test_data_pd.nunique()

viewer_id             418020
date                       5
tv_make                    4
tv_size                    8
uhd_capable                2
tv_provider                5
total_time_watched        95
watched                    2
test                       2
dtype: int64

In [13]:
test_data_pd.describe()

,viewer_id,tv_size,uhd_capable,total_time_watched,watched,test
count,4.180260e+05,418026.000000,418026.000000,418026.000000,418026.000000,418026.000000
mean,1.835585e+07,51.874111,0.200858,10.039700,0.054547,0.488790
std,1.248317e+07,12.225413,0.400643,6.179722,0.227094,0.499875
min,1.000000e+04,32.000000,0.000000,0.250000,0.000000,0.000000
25%,7.526723e+06,40.000000,0.000000,5.000000,0.000000,0.000000
50%,1.838327e+07,55.000000,0.000000,9.500000,0.000000,0.000000
75%,2.916353e+07,65.000000,0.000000,14.500000,0.000000,1.000000
max,3.999992e+07,70.000000,1.000000,23.750000,1.000000,1.000000


#### How many dates there are?

In [14]:
date_list = pd.date_range(test_data_pd.date.min(), test_data_pd.date.max())
date_list

DatetimeIndex(['2018-01-15', '2018-01-16', '2018-01-17', '2018-01-18',
               '2018-01-19'],
              dtype='datetime64[ns]', freq='D')

In [15]:
len(date_list)

5

In [16]:
test_data_pd.shape

(418026, 9)

In [17]:
test_data_pd.count().sum()

3709514

### How many nulls in test_data?

In [18]:
# total
test_data_pd.isnull().sum().sum()

52720

In [19]:
# which column?
test_data_pd.isnull().any()

viewer_id             False
date                  False
tv_make               False
tv_size               False
uhd_capable           False
tv_provider            True
total_time_watched    False
watched               False
test                  False
dtype: bool

In [20]:
# how many nulls in the column tv_provider?
test_data_pd.isnull().sum()

viewer_id                 0
date                      0
tv_make                   0
tv_size                   0
uhd_capable               0
tv_provider           52720
total_time_watched        0
watched                   0
test                      0
dtype: int64

### How many nulls in viewer_data?

In [21]:
# total
viewer_data_pd.isnull().sum().sum()

0

#### let's convert to PySpark for more convenient data wrangling

In [22]:
test_schema = StructType(
    [
        StructField("viewer_id", StringType(), True),
        StructField("date", StringType(), True),
        StructField("tv_make", StringType(), True),
        StructField("tv_size", StringType(), True),
        StructField("uhd_capable", IntegerType(), True),
        StructField("tv_provider", StringType(), True),
        StructField("total_time_watched", DoubleType(), True),
        StructField("watched", IntegerType(), True),
        StructField("test", IntegerType(), True)
    ]
)

In [23]:
test_data_df = spark.createDataFrame(test_data_pd, schema=test_schema)
test_data_df.printSchema()

root
 |-- viewer_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- tv_make: string (nullable = true)
 |-- tv_size: string (nullable = true)
 |-- uhd_capable: integer (nullable = true)
 |-- tv_provider: string (nullable = true)
 |-- total_time_watched: double (nullable = true)
 |-- watched: integer (nullable = true)
 |-- test: integer (nullable = true)



In [24]:
# not as pretty as Pandas, I know ;)
test_data_df.show(2)

21/08/19 00:41:25 WARN TaskSetManager: Stage 0 contains a task of very large size (3256 KiB). The maximum recommended task size is 1000 KiB.


+---------+----------+-------+-------+-----------+-----------+------------------+-------+----+
|viewer_id|      date|tv_make|tv_size|uhd_capable|tv_provider|total_time_watched|watched|test|
+---------+----------+-------+-------+-----------+-----------+------------------+-------+----+
| 24726768|2018-01-16|   Sony|     70|          0|    Comcast|             10.75|      0|   1|
| 25001464|2018-01-18|   Sony|     32|          0|        NaN|              2.75|      0|   0|
+---------+----------+-------+-------+-----------+-----------+------------------+-------+----+
only showing top 2 rows



In [25]:
viewer_schema = StructType(
    [
        StructField("viewer_id", StringType(), True),
        StructField("gender", StringType(), True),
        StructField("age", IntegerType(), True),
        StructField("city", StringType(), True)
    ]
)

In [26]:
viewer_data_df = spark.createDataFrame(viewer_data_pd, schema=viewer_schema)
viewer_data_df.printSchema()

root
 |-- viewer_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- city: string (nullable = true)



In [27]:
# join of both tables
test_data_with_viewer_df = (test_data_df
                            .join(other=viewer_data_df, on='viewer_id', how='left')
                           )

In [28]:
# did I find all viewers in dictionary?
(test_data_with_viewer_df
 .groupBy('gender')
 .count()
).show(10, False)

21/08/19 00:41:28 WARN TaskSetManager: Stage 1 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:41:29 WARN TaskSetManager: Stage 2 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.


+------+------+
|gender|count |
+------+------+
|null  |556   |
|Female|209420|
|Male  |208050|
+------+------+



- some viewers are not in dictionary
- let's see how all nulls affect experiment

### How nulls affect experiment?

In [29]:
(test_data_with_viewer_df
 .withColumn('no_viewer_data', f.col('gender').isNull())
 .withColumn('no_tv_provider', f.col('tv_provider').isNull())
 .groupBy('test')
 .agg(f.count('*').alias('sample_size'),
      f.count(f.when(~f.col('no_viewer_data'), '*')).alias('sample_size_with_dict')
     )
).show(10, False)

21/08/19 00:41:34 WARN TaskSetManager: Stage 21 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:41:35 WARN TaskSetManager: Stage 22 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.


+----+-----------+---------------------+
|test|sample_size|sample_size_with_dict|
+----+-----------+---------------------+
|1   |204327     |204327               |
|0   |213699     |213143               |
+----+-----------+---------------------+



In [30]:
(213699-204327)/213699

0.04385607794140356

- difference in sample size accounts for 4.4%. That is a lot.
- but I still need to check one thing: city of viewer. It may change something.

In [31]:
(test_data_with_viewer_df
 .withColumn('no_viewer_data', f.col('gender').isNull())
 .withColumn('no_tv_provider', f.col('tv_provider').isNull())
 .withColumn('city_LA', f.col('city') == 'Los Angeles')
 .groupBy('test', 'city_LA')
 .agg(f.count('*').alias('sample_size'),
      f.count(f.when(~f.col('no_viewer_data'), '*')).alias('sample_size_with_dict')
     )
 .orderBy('test', 'city_LA')
).show(10, False)

21/08/19 00:41:38 WARN TaskSetManager: Stage 41 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:41:39 WARN TaskSetManager: Stage 42 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.


+----+-------+-----------+---------------------+
|test|city_LA|sample_size|sample_size_with_dict|
+----+-------+-----------+---------------------+
|0   |null   |556        |0                    |
|0   |false  |160630     |160630               |
|0   |true   |52513      |52513                |
|1   |false  |204327     |204327               |
+----+-------+-----------+---------------------+



In [32]:
(test_data_with_viewer_df
 .withColumn('no_viewer_data', f.col('gender').isNull())
 .withColumn('no_tv_provider', f.col('tv_provider').isNull())
 .withColumn('city_LA', f.col('city') == 'Los Angeles')
 .groupBy('city', 'test')
 .agg(f.count('*').alias('sample_size'),
      f.count(f.when(~f.col('no_viewer_data'), '*')).alias('sample_size_with_dict')
     )
 .orderBy('city', 'test')
).show(50, False)

21/08/19 00:41:42 WARN TaskSetManager: Stage 45 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:41:42 WARN TaskSetManager: Stage 46 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.


+-------------+----+-----------+---------------------+
|city         |test|sample_size|sample_size_with_dict|
+-------------+----+-----------+---------------------+
|null         |0   |556        |0                    |
|Atlanta      |0   |11728      |11728                |
|Atlanta      |1   |12599      |12599                |
|Boston       |0   |11758      |11758                |
|Boston       |1   |12660      |12660                |
|Chicago      |0   |16054      |16054                |
|Chicago      |1   |16991      |16991                |
|Dallas       |0   |12874      |12874                |
|Dallas       |1   |13807      |13807                |
|Detroit      |0   |8739       |8739                 |
|Detroit      |1   |9192       |9192                 |
|Houston      |0   |11802      |11802                |
|Houston      |1   |12589      |12589                |
|Los Angeles  |0   |52513      |52513                |
|Miami        |0   |8301       |8301                 |
|Miami    

#### The table above tells us sth about experiment setup.

##### Key findings:
- both group should be equally distributed regarding charactersics of users. There are no users from Los Angeles in the test group. There is also huge underrepresentation of Seattle in control group. Thus, there is a risk that we compare groups that are not evenly distributed.

### 1. Reproduce the negative result by Channel KLMN's data scientist
- I assume that nulls were included by Channel KLMN's data scientist

In [33]:
window_spec = (Window
               .orderBy('test')
              )
test_data_agg = (test_data_with_viewer_df
                 .groupBy('test')
                 .agg(f.sum(f.col('watched')).alias('watched'),
                      f.count('*').alias('all_userdays')
                     )
                 .withColumn('conversion', f.col('watched')/f.col('all_userdays'))
                 .withColumn('control_group_value', f.first(f.col('conversion')).over(window_spec))
                 .withColumn('diff_to_base_pp', 100*(f.col('conversion') - f.col('control_group_value')))
                 .withColumn('diff_to_base_perc', (f.col('conversion') - f.col('control_group_value'))/f.col('control_group_value'))
                )

In [34]:
test_data_agg.orderBy('test').show(10, False)

21/08/19 00:41:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/08/19 00:41:46 WARN TaskSetManager: Stage 49 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:41:47 WARN TaskSetManager: Stage 50 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.


+----+-------+------------+-------------------+-------------------+-------------------+--------------------+
|test|watched|all_userdays|conversion         |control_group_value|diff_to_base_pp    |diff_to_base_perc   |
+----+-------+------------+-------------------+-------------------+-------------------+--------------------+
|0   |13448  |213699      |0.0629296346730682 |0.0629296346730682 |0.0                |0.0                 |
|1   |9354   |204327      |0.04577955923593064|0.0629296346730682 |-1.7150075437137555|-0.27252780865860043|
+----+-------+------------+-------------------+-------------------+-------------------+--------------------+



- we can already see a huge negative difference (-27%)
- probably no significance test required but let's perform it

#### First, let's check if the sample size is big enough to assure the difference.

In [35]:
import math
import scipy
def get_minimum_detectable_impact(sample_size, metric_avg, metric_std=0, metric='binary', alfa=0.05):

    z_a = scipy.stats.norm.ppf(1-(alfa/2)) # Z alpha
    z_b = scipy.stats.norm.ppf(1-alfa) # Z test power
    
    if metric == 'binary':
        min_impact = math.sqrt((2 * (z_a + z_b) ** 2 * metric_avg * (1 - metric_avg)) / (sample_size * metric_avg ** 2))
    else:
        min_impact = math.sqrt((2 * (z_a + z_b) ** 2 * metric_std ** 2) / (sample_size * metric_avg ** 2))
    return min_impact

In [36]:
get_minimum_detectable_impact(sample_size=213699,
                              metric_avg=0.0629296346730682,
                             )

0.04255548510670759

#### The difference is big enough to perform significance test.

In [37]:
test_results = []
def getStats(df, cols, alfa=0.05):

    pd_df = df.drop('clicked_cnt', 'non_clicked_cnt').toPandas().set_index(['test'])
    
    pd_df.at[[(0)], 'significant'] = False
    sample_size=(df
            .where(f.col('test') == 0)
                .collect()[0].all_userdays)
    metric_avg=(df
            .where(f.col('test') == 0)
                .collect()[0].conversion)
    print('sample_size:', sample_size, " metric_avg:", metric_avg)
    minimum_detectable_impact = get_minimum_detectable_impact(sample_size=sample_size, metric_avg=metric_avg, alfa=alfa)
    print('get_minimum_detectable_impact:', minimum_detectable_impact)

    tmp = (df
        .where(f.col('test').isin([0, 1]))
        .select(['test']+cols))
    diff=abs(tmp.where(f.col('test')== 1).collect()[0].conversion
          -tmp.where(f.col('test')== 0 ).collect()[0].conversion)/tmp.where(f.col('test')== 0).collect()[0].conversion
    obs = np.array(tmp.select(cols[:2]).limit(2000000).collect())
    try:
        p = chi2_contingency(obs)[1]
        significant = (p<alfa) & (diff>=minimum_detectable_impact)
        pd_df.at[[(1)], 'significant'] = significant
    except ValueError:
        pass    
         
    return pd_df
    return (pd_df.sort_values(by=['test'], axis='index')
            [['all_userdays', 'conversion', 'diff_to_base_pp', 'diff_to_base_perc', 'significant']]
            .style.format({'conversion': "{:.2%}",'diff_to_base_pp': "{:.2f} pp",'diff_to_base_perc': "{:.2%}"})).set_caption('Konwersja na SI: wszystkie listingi')

In [38]:
test_data_agg_chi = (test_data_agg
                     .withColumn('not_watched', f.col('all_userdays')-f.col('watched'))
                    )
test_data_agg_chi.show(1, False, True)

21/08/19 00:41:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/08/19 00:41:50 WARN TaskSetManager: Stage 54 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:41:51 WARN TaskSetManager: Stage 55 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0---------------------------------
 test                | 0                  
 watched             | 13448              
 all_userdays        | 213699             
 conversion          | 0.0629296346730682 
 control_group_value | 0.0629296346730682 
 diff_to_base_pp     | 0.0                
 diff_to_base_perc   | 0.0                
 not_watched         | 200251             
only showing top 1 row



In [39]:
cols_chi = ['watched', 'not_watched', 'conversion' ,'diff_to_base_pp', 'diff_to_base_perc']

In [40]:
result_pd = getStats(df=test_data_agg_chi.drop('control_group_value'),
                     cols=cols_chi,
                     alfa = 0.05
                    )

21/08/19 00:41:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/08/19 00:41:54 WARN TaskSetManager: Stage 59 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:41:54 WARN TaskSetManager: Stage 60 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:41:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/08/19 00:41:57 WARN TaskSetManager: Stage 64 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:41:57 WARN TaskSetManager: Stage 65 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a sin

sample_size: 213699  metric_avg: 0.0629296346730682
get_minimum_detectable_impact: 0.04255548510670759


21/08/19 00:42:03 WARN TaskSetManager: Stage 75 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/08/19 00:42:05 WARN TaskSetManager: Stage 79 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:06 WARN TaskSetManager: Stage 80 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/08/19 00:42:08 WARN TaskSetManager: Stage 84 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:09 WARN TaskSetManager: Stage 85 contains a task of very large size (3255 KiB). The max

In [41]:
(result_pd[['all_userdays', 'conversion', 'diff_to_base_pp', 'diff_to_base_perc', 'significant']]
.style.format({'conversion': "{:.2%}",'diff_to_base_pp': "{:.2f} pp",'diff_to_base_perc': "{:.2%}"})
).set_caption('Local mayor ad conversion') 

,all_userdays,conversion,diff_to_base_pp,diff_to_base_perc,significant
test,,,,,
0,213699,6.29%,0.00 pp,0.00%,False
1,204327,4.58%,-1.72 pp,-27.25%,True


#### Answer 1: The result is negative (significant difference).

### 2. Investigate what might be happening.  Are the commercials with local Mayors really driving a lower fraction of people to watch the show?

#### My hypothesis is that uneven distribution of users (by city) drives this difference.
Let's check!

In [42]:
window_spec_city = Window.partitionBy('city').orderBy('test')

test_data_city_agg = (test_data_with_viewer_df
                      .groupBy('city', 'test')
                      .agg(f.sum(f.col('watched')).alias('watched'),
                           f.count('*').alias('all_userdays')
                          )
                      .withColumn('conversion', f.col('watched')/f.col('all_userdays'))
                      .withColumn('control_group_value', f.first(f.col('conversion')).over(window_spec_city))
                      .withColumn('control_group_sample_size', f.first(f.col('all_userdays')).over(window_spec_city))
                      .withColumn('diff_to_base_perc', (f.col('conversion') - f.col('control_group_value')) /
                                  f.col('control_group_value'))
                      .withColumn('diff_to_base_perc_sample_size', (f.col('all_userdays') - f.col('control_group_sample_size')) /
                                  f.col('control_group_sample_size'))
                      .orderBy('city', 'test')
                     )

In [43]:
test_data_city_agg.toPandas()\
.style.format({'conversion': "{:.2%}",
                'diff_to_base_pp': "{:.2f} pp",
                'diff_to_base_perc': "{:.2%}"})

21/08/19 00:42:14 WARN TaskSetManager: Stage 109 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:15 WARN TaskSetManager: Stage 110 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.


,city,test,watched,all_userdays,conversion,control_group_value,control_group_sample_size,diff_to_base_perc,diff_to_base_perc_sample_size
0,None,0,0,556,0.00%,0.000000,556,nan%,0.000000
1,Atlanta,0,589,11728,5.02%,0.050222,11728,0.00%,0.000000
2,Atlanta,1,641,12599,5.09%,0.050222,11728,1.30%,0.074267
3,Boston,0,605,11758,5.15%,0.051454,11758,0.00%,0.000000
4,Boston,1,662,12660,5.23%,0.051454,11758,1.63%,0.076714
5,Chicago,0,794,16054,4.95%,0.049458,16054,0.00%,0.000000
6,Chicago,1,878,16991,5.17%,0.049458,16054,4.48%,0.058366
7,Dallas,0,655,12874,5.09%,0.050878,12874,0.00%,0.000000
8,Dallas,1,660,13807,4.78%,0.050878,12874,-6.05%,0.072472
9,Detroit,0,410,8739,4.69%,0.046916,8739,0.00%,0.000000


In [44]:
(test_data_city_agg
 .where(f.col('test') == 1)
 .drop('control_group_value', 'watched')
 .orderBy('diff_to_base_perc')
 .toPandas()
 .style.format({'conversion': "{:.2%}",
                'diff_to_base_pp': "{:.2f} pp",
                'diff_to_base_perc': "{:.2%}",
                'diff_to_base_perc_sample_size': "{:.2%}"})
)

21/08/19 00:42:19 WARN TaskSetManager: Stage 120 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:19 WARN TaskSetManager: Stage 121 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.


,city,test,all_userdays,conversion,control_group_sample_size,diff_to_base_perc,diff_to_base_perc_sample_size
0,Seattle,1,15850,2.57%,2846,-8.65%,456.92%
1,Dallas,1,13807,4.78%,12874,-6.05%,7.25%
2,New York,1,35812,5.00%,34082,-5.73%,5.08%
3,Phoenix,1,10015,5.15%,9317,-3.99%,7.49%
4,Minneapolis,1,9093,5.25%,8203,-1.30%,10.85%
5,San Francisco,1,12609,4.96%,11691,-0.61%,7.85%
6,Philadelphia,1,24495,2.35%,4261,0.02%,474.87%
7,Atlanta,1,12599,5.09%,11728,1.30%,7.43%
8,Boston,1,12660,5.23%,11758,1.63%,7.67%
9,Tampa,1,9631,5.20%,8974,2.15%,7.32%


### Answer 2a: Commercials with local Mayors may help but this is highly dependent on city. It may be that cities with negative results have unpopular Mayors (just assumption).
As we can see there is huge misrepresentation for 3 cities and this probably causes the difference:
- Los Angeles (missing in test group)
- Seattle (overrepresentation in test group)
- Philadelphia (overrepresentation in test group)

#### Let's perform test once again without cities mentioned above.

In [45]:
window_spec = (Window
               .orderBy('test')
              )
test_data_agg_v2 = (test_data_with_viewer_df
                    .where(~f.col('city').isin('Philadelphia', 'Los Angeles', 'Seattle'))
                    .groupBy('test')
                    .agg(f.sum(f.col('watched')).alias('watched'),
                         f.count('*').alias('all_userdays')
                        )
                    .withColumn('conversion', f.col('watched')/f.col('all_userdays'))
                    .withColumn('control_group_value', f.first(f.col('conversion')).over(window_spec))
                    .withColumn('diff_to_base_pp', 100*(f.col('conversion') - f.col('control_group_value')))
                    .withColumn('diff_to_base_perc', (f.col('conversion') - f.col('control_group_value'))/f.col('control_group_value'))
                )

In [46]:
test_data_agg_chi_v2 = (test_data_agg_v2
                        .withColumn('not_watched', f.col('all_userdays')-f.col('watched'))
                       )

In [47]:
result_pd_v2 = getStats(df=test_data_agg_chi_v2.drop('control_group_value'),
                     cols=cols_chi,
                     alfa = 0.05
                    )

21/08/19 00:42:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/08/19 00:42:23 WARN TaskSetManager: Stage 131 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:23 WARN TaskSetManager: Stage 132 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/08/19 00:42:25 WARN TaskSetManager: Stage 136 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:26 WARN TaskSetManager: Stage 137 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a

sample_size: 153523  metric_avg: 0.05117148570572488
get_minimum_detectable_impact: 0.05602623265979933


21/08/19 00:42:31 WARN TaskSetManager: Stage 147 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/08/19 00:42:33 WARN TaskSetManager: Stage 151 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:33 WARN TaskSetManager: Stage 152 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/08/19 00:42:35 WARN TaskSetManager: Stage 156 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:35 WARN TaskSetManager: Stage 157 contains a task of very large size (3255 KiB). Th

In [48]:
(result_pd_v2[['all_userdays', 'conversion', 'diff_to_base_pp', 'diff_to_base_perc', 'significant']]
.style.format({'conversion': "{:.2%}",'diff_to_base_pp': "{:.2f} pp",'diff_to_base_perc': "{:.2%}"})
).set_caption('Local mayor ad conversion') 

,all_userdays,conversion,diff_to_base_pp,diff_to_base_perc,significant
test,,,,,
0,153523,5.12%,0.00 pp,0.00%,False
1,163982,5.11%,-0.01 pp,-0.23%,False


### Answer 2b: As we can see, removal of unevenly distributed cities shows there is no difference. Therefore we can blame uneven distribution for observed differences.

### 3. If you found something wrong with the experiment, write pseudocode for an algorithm that returns FALSE if that problem happens again in the future and TRUE otherwise.  This must be either implemented as a function or written as pseudocode.

### If you didn’t find anything wrong with the experiment, what is your recommendation to the Executive Producer of “US Politics This Week” regarding whether or not they should continue airing the new commercials?

- I spotted difference in samples of cities but let's assume we want to check if also other features were evenly distributed.

In [49]:
def check_if_even_distribution(df, features, experiment_variants_number, accepted_threshold):
    diffs_list = []
    
    for feature in features:
        
        window_spec_check = Window.partitionBy(feature).orderBy('test')
        
        # for difference in count between variants
        diff_count = (df
                      .where(f.col(feature).isNotNull())
                      .groupBy(feature, 'test')
                      .agg(f.count('*').alias('count'))
                      .withColumn('first_variant_count', f.first(f.col('count')).over(window_spec_check))
                      .withColumn('diff_to_first', f.abs(f.col('count') - f.col('first_variant_count')) /
                                  f.col('first_variant_count'))
                      .withColumn('unevenly_distributed', f.col('diff_to_first') > accepted_threshold)
                      .where(f.col('unevenly_distributed'))
                      .count()
                     )
        diffs_list.append(diff_count)
        
        # for lack of this variant of the feature (e.g. triggers on Los Angeles not being in control group)
        variants_missed = (df
                           .where(f.col(feature).isNotNull())
                           .groupBy(feature)
                           .agg(f.countDistinct(f.col('test')).alias('variants_cnt'))
                           .where(f.col('variants_cnt') < experiment_variants_number)
                          ).count()
        diffs_list.append(variants_missed)
    return sum(diffs_list) == 0

In [50]:
# should return False
check_if_even_distribution(df=test_data_with_viewer_df,
                           features=test_data_with_viewer_df.columns[-3:],
                           experiment_variants_number = 2,
                           accepted_threshold = 0.1
                          )

21/08/19 00:42:40 WARN TaskSetManager: Stage 181 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:40 WARN TaskSetManager: Stage 182 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:43 WARN TaskSetManager: Stage 187 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:44 WARN TaskSetManager: Stage 188 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:47 WARN TaskSetManager: Stage 193 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:47 WARN TaskSetManager: Stage 194 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:42:51 WARN TaskSetManager: Stage 199 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.

False

In [51]:
# should return True
test_data_with_viewer_clean_df = (test_data_with_viewer_df
                                  .where(~f.col('city').isin('Philadelphia', 'Los Angeles', 'Seattle', 'Minneapolis'))
                                 )

check_if_even_distribution(df=test_data_with_viewer_clean_df,
                           features=['city'],
                           experiment_variants_number = 2,
                           accepted_threshold = 0.1
                          )

21/08/19 00:43:00 WARN TaskSetManager: Stage 217 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:43:01 WARN TaskSetManager: Stage 218 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:43:03 WARN TaskSetManager: Stage 223 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.
21/08/19 00:43:04 WARN TaskSetManager: Stage 224 contains a task of very large size (3255 KiB). The maximum recommended task size is 1000 KiB.


True

#### Answer 3: The above function returns False if samples are unevenly distributed and True if samples are in similar shape.

Can be used for:
- predefined column names
- all number of variants (also multi-varaite expeeriments)
- different levels of sample size difference acceptance